# Applying BespokeFit to the pop-terphenyl system

This notebook will be used to generate OpenFF force field for the pop-terphenyl system. We use mBuild to construct a short 3-mer of the polymer and get it's charges using AM1-BCC charges using OpenEye. Then using the BespokeFit workflow we will assign torsions/angles. Bespoke fit uses QM calculations on fragments present in our molecule to assign new more accurate torsion potentials.

In [1]:
try:
    from openmm import app
except ImportError:
    from simtk.openmm import app

from openff.toolkit.topology import FrozenMolecule, Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import pdb
import os
import openbabel
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer
import rdkit

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy

In [2]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)(c3ccc(cc3)[C@H](N)C))cc1', smiles = True, name="POP")
cap_o = mb.load('CO', smiles = True)
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2863 -0.53   -0.1319]), 1 bonds, id: 140325421845712>
26 <H pos=([-0.3535 -0.356   0.0328]), 1 bonds, id: 140325421845952>
27 <H pos=([-0.3417 -0.1275  0.1218]), 1 bonds, id: 140325421846192>
28 <H pos=([0.0238 0.3237 0.1286]), 1 bonds, id: 140325421846432>
29 <H pos=([-0.2587  0.6644  0.2184]), 1 bonds, id: 140325421846480>
30 <H pos=([-0.3616  0.5219  0.2592]), 1 bonds, id: 140325421863360>
31 <H pos=([-0.3482  0.5794  0.0869]), 1 bonds, id: 140325421863600>
32 <H pos=([-0.4049  0.3384  0.0985]), 1 bonds, id: 140325421863840>
33 <H pos=([-0.4091  0.1083  0.0184]), 1 bonds, id: 140325421864080>
34 <H pos=([ 0.1529  0.2134 -0.0776]), 1 bonds, id: 140325421864320>
35 <H pos=([ 0.365   0.0909 -0.0926]), 1 bonds, id: 140325421864560>
36 <H pos=([ 0.2052 -0.2164  0.1649]), 1 bonds, id: 140325421864800>
37 <H pos=([-0.006  -0.093   0.1809]), 1 bonds, id: 140325421865040>
38 <H pos=([ 0.4333 -0.2302  0.1013]), 1 bonds, id: 140325421865280>
39 <H pos=([ 0.6328 -0.1197  0.0441])

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2232 -0.4572 -0.1813]), 3 bonds, id: 140325421958528> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.1572 -0.4926 -0.2826]), 1 bonds, id: 140325421958624> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2155 -0.3192 -0.1298]), 3 bonds, id: 140325421958816> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[25, 40],
                  separation=.15,
                  replace=True,
                  # orientation = [[0,-1,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=3, sequence='A')

In [6]:
# Set residue labels for polymer
for label in chain.labels["monomer"]:
    label.name = "TRI"
for label in chain.labels["Compound"]:
    label.name = "CAP"


In [7]:
# Rename atoms using mbuild interface
counts = {}
for particle in chain.particles():
    atom_name = particle.name
    if not atom_name in counts.keys():
        counts[atom_name] = 1
    else:
        counts[atom_name] += 1
    particle.name = atom_name + str(counts[atom_name])

In [8]:
# Write to PDB
chain.save("pop_trimer_mbuild.pdb", overwrite = True, residues = ["TRI", "CAP"])

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C1 pos=([-0.2232 -0.4572 -0.1813]), 3 bonds, id: 140325421254304> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O1 pos=([-0.1572 -0.4926 -0.2826]), 1 bonds, id: 140325194229936> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C2 pos=([-0.2155 -0.3192 -0.1298]), 3 bonds, id: 140325194230032> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/si

# File conversion between `mbuild` and OpenFF

Ideally structures generated from `mbuild` could be directly imported into the OpenFF workflow, however they are not. We have to recover connectivity information and write structures out into `.mol` and `.pdb` files to put into the OpenFF forcefield assignment workflow.

In [9]:
# Load into RDKit to assign bond infomration in PDB file
rdmol = rdkit.Chem.rdmolfiles.MolFromPDBFile("pop_trimer_mbuild.pdb", removeHs=False)
for atom in rdmol.GetAtoms():
    ri = atom.GetPDBResidueInfo()
    ri.SetIsHeteroAtom(False)
rdkit.Chem.rdmolfiles.MolToPDBFile(rdmol, "pop_trimer_mbuild_bonds.pdb")

In [10]:
# OpenBabel to convert PDB to Mol format
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("pdb", "mol")
mol = openbabel.OBMol()
obConversion.ReadFile(mol, "pop_trimer_mbuild_bonds.pdb")
obConversion.WriteFile(mol, "pop_trimer_mbuild.mol")

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


True

# OpenFF Parameter Assignment

Now that we have our input `.mol` and `.pdb` files we can assign bonded and non-bonded parameters using OpenFF.

In [11]:
# Load in with OpenFF
tri_pop = Molecule.from_file("pop_trimer_mbuild.mol")

In [12]:
pdbfile = app.PDBFile("pop_trimer_mbuild_bonds.pdb")
omm_topology = pdbfile.topology
omm_topology

<Topology; 1 chains, 5 residues, 153 atoms, 161 bonds>

In [13]:
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[tri_pop]
)

In [14]:
from openff.toolkit.utils.toolkits import GLOBAL_TOOLKIT_REGISTRY as tkr
from openff.toolkit.utils.toolkits import OpenEyeToolkitWrapper, RDKitToolkitWrapper, AmberToolsToolkitWrapper

# Deregister other toolkits to make sure we use OpenEye's
# implementation of AM1-BCC charging

tkr.deregister_toolkit(RDKitToolkitWrapper)
tkr.deregister_toolkit(AmberToolsToolkitWrapper)



In [15]:
# Modified OpenFF to increase maxAtoms for AM1BCC method to 500 atoms
if not os.path.exists('terphenyl_pop_trimer_charges.sdf'):
    tri_pop.assign_partial_charges(partial_charge_method="am1bcc")
    tri_pop.to_file('terphenyl_pop_trimer_charges.sdf', file_format = 'sdf')
else:
    tri_pop = Molecule.from_file('terphenyl_pop_trimer_charges.sdf')

In [16]:
tkr.register_toolkit(RDKitToolkitWrapper)
tkr.register_toolkit(AmberToolsToolkitWrapper)

# Submit to BespokeFit Executor

The BespokeFit workflow needs an SDF file with charges to find fragments and assign a bespoke set of force-field parameters for chemical systems. Make sure you have an instance of the Bespoke fit executor running in another terminal by running:

```
BEFLOW_OPTIMIZER_KEEP_FILES=True openff-bespoke executor launch \
    --n-fragmenter-workers 1                                    \
    --n-optimizer-workers  1                                    \
    --n-qc-compute-workers 1                                    \
    --qc-compute-n-cores   1                                    \
    --qc-compute-max-mem   1.5                                  \
    --directory            bespoke-executor
```

These flags determine how BespokeFit distributes processes to perform the bespokefit. The `BEFLOW_OPTIMIZER_KEEP_FILES=True` allows us to look at output files from the fit.

In [17]:
!BEFLOW_OPTIMIZER_KEEP_FILES=True openff-bespoke executor submit \
     --file                 "terphenyl_pop_trimer_charges.sdf" \
     --workflow             "default"                            \
     --default-qc-spec      xtb gfn2xtb none


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

1. preparing the bespoke workflow                                               
                                                                                
⠋ loading the molecules
[✓] 1 molecules found
building fitting schemas ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
[✓] fitting schemas generated
                                                                                
2. submitting the workflow                                                      
                                                                                
submitting tasks ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
[✓] the following workflows were submitted
┏━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ SMILES            ┃ NAME             ┃ FILE                             ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩


In [23]:
!openff-bespoke executor list


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

The following optimizations were found:
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ ID ┃ SMILES                                                        ┃ STATUS  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ 1  │ C[C@H](c1cccc(c1)c2cc(ccc2c3ccc(cc3)C(=O)N[C@H](C)c4cccc(c4)c │ success │
│    │ 5cc(ccc5c6ccc(cc6)C(=O)OC)OC)OC)NC(=O)c7ccc(cc7)c8ccc(cc8c9cc │         │
│    │ cc(c9)[C@@H](C)NC(=O)OC(C)(C)C)OC                             │         │
│ 2  │ C[C@@H](c1ccc(cc1)c2cc(ccc2c3ccc(cc3)C(=O)N[C@@H](C)c4ccc(cc4 │ success │
│    │ )c5cc(ccc5c6ccc(cc6)C(=O)OC)OC)OC)NC(=O)c7ccc(cc7)c8ccc(cc8c9 │         │
│    │ ccc(cc9)[C@H](C)NC(=O)OC(C)(C)C)OC                            │         │
│ 3  │ C[C@H](c1ccc(cc1)c2cc(ccc2c3ccc(cc3)C(=O)N[C@H](C)c4ccc(cc4)c │ success │
│    │ 5cc(ccc5c6ccc(cc6)C(=O)OC)OC)OC)NC(=O)c7ccc(cc7)c8ccc(cc8c9c

In [21]:
!openff-bespoke executor watch --id 3


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

⠋ fragmenting the moleculele
[✓] fragmentation successful
⠦ generating bespoke QC datata
[✓] qc-generation successful
⠴ optimizing the parametersrs
[✓] optimization successful


In [22]:
!openff-bespoke executor retrieve --id 3 --output "pop_trimer_bespoke_fit.json" --force-field "openff-2.0.0_bespoke_pop_trimer.offxml"


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

[✓] the bespoke fit is finished
                                                                                
outputs have been saved to pop_trimer_bespoke_fit.json                          
                                                                                
                                                                                
the bespoke force field has been saved to openff-2.0.0_bespoke_pop_trimer.offxml
                                                                                
